In [1]:
from utils import *
import pandas as pd
from m3inference import M3Twitter
from m3inference.utils import transform_jsonl_object_for_m3inference_text_model
from googletrans import Translator
TRANSLATOR = Translator()

In [2]:
data = read_jsonl('test.json')
users = [d['user'] for d in data]

In [12]:
new_data = []
with open('test_processed_0.json', 'w') as f:
    for tweet in data:
        new_tweet = {}
        new_tweet['lang'] = tweet['lang']
        new_tweet['id_str'] = tweet['id_str']
        new_tweet['text_original'] = clean_tweet_text(get_tweet_text(tweet))
        new_user = {}
        new_user['lang'] = tweet['user']['lang']
        new_user['id'] = tweet['user']['id_str']
        new_user['name'] = tweet['user']['name']
        new_user['screen_name'] = tweet['user']['screen_name']
        new_user['location_original'] = tweet['user']['location']
        new_user['description_original'] = tweet['user']['description']
  
        new_tweet['user'] = new_user

        new_data.append(new_tweet)
        
        f.write(json.dumps(new_tweet) + '\n')

In [15]:
with open('test_processed_1.json', 'w') as f:
    for new_tweet in new_data:
  
        if new_tweet['lang'] != 'en':

            text_translated = TRANSLATOR.translate(new_tweet['text_original'], dest='en')
            new_tweet['text_english'] = text_translated.text  
            new_tweet['text_lang_detected'] = text_translated.src  
            
        else: 
            new_tweet['text_english'] = new_tweet['text_original']
            new_tweet['text_lang_detected'] = 'en'
        
        f.write(json.dumps(new_tweet) + '\n')
# 8m 35.7s

In [16]:
with open('test_processed_2.json', 'w') as f:
    for new_tweet in new_data:
        
        new_user = new_tweet['user']

        if new_user['lang'] != 'en':
            
            location_translated = TRANSLATOR.translate(new_user['location_original'], dest='en')

            new_user['location_english'] = location_translated.text
            new_user['location_lang_detected'] = location_translated.src

        else:
            new_user['location_english'] = new_user['location_original']
            new_user['location_lang_detected'] = 'en'
            new_user['description_english'] = new_user['description_original']
            new_user['description_lang_detected'] = 'en'
        
        f.write(json.dumps(new_tweet) + '\n')
# 11m 41.2s

In [17]:
with open('test_processed_2.json', 'w') as f:
    for new_tweet in new_data:
        
        new_user = new_tweet['user']

        if new_user['lang'] != 'en':

            description_translated = TRANSLATOR.translate(new_user['description_original'], dest='en')
            new_user['description_english'] = description_translated.text
            new_user['description_lang_detected'] = description_translated.src

        
        f.write(json.dumps(new_tweet) + '\n')
# 12m 35.2s

In [21]:
with open('test_translated.json', 'w') as f:
    for new_tweet in new_data:        
        f.write(json.dumps(new_tweet) + '\n')

In [72]:
for new_tweet in new_data:
    new_user = new_tweet["user"]
    new_user["text_lang_detected"] = new_tweet["text_lang_detected"]
    new_user["text_lang"] = new_tweet["lang"]

new_users = [new_tweet["user"] for new_tweet in new_data]

new_users_dict = {}
for new_user in new_users:
    user_id = new_user["id"]
    if user_id not in new_users_dict:
        new_users_dict[user_id] = {}
        for k, v in new_user.items():
            if k != "id":
                new_users_dict[user_id][k] = [v]
            else:
                new_users_dict[user_id][k] = v
    else:
        for k, v in new_user.items():
            if v not in new_users_dict[user_id][k]:
                new_users_dict[user_id][k].append(v)

new_users = [new_users_dict[user_id] for user_id in new_users_dict]

In [74]:
[user for user in new_users if len(user["text_lang_detected"]) > 1]

[{'lang': [None],
  'id': '1014342437936902146',
  'name': ['WANARAT♥️วอร์=ความสุข🤟🏻'],
  'screen_name': ['PJSS06'],
  'location_original': ['Thailand'],
  'description_original': ['วอร์คือที่1ในใจ @warwanarat #warwanarat'],
  'location_english': ['Thailand'],
  'location_lang_detected': ['en'],
  'description_english': ['War is number 1 in my heart @warwanarat #warwanarat'],
  'description_lang_detected': ['th'],
  'text_lang_detected': ['th', 'en'],
  'text_lang': ['th', 'en']},
 {'lang': [None],
  'id': '1175077409319612417',
  'name': ['Rooster'],
  'screen_name': ['mkamarul_'],
  'location_original': ['Malaysia'],
  'description_original': ['YNWA❤️'],
  'location_english': ['Malaysia'],
  'location_lang_detected': ['en'],
  'description_english': ['YNWA❤️'],
  'description_lang_detected': ['lus'],
  'text_lang_detected': ['ms', 'en'],
  'text_lang': ['in', 'en']}]

In [ ]:
new_users = list(new_users_dict.values())

In [63]:
[user for user in new_users if user['lang'][0] != None]

[]

In [64]:
m3twitter=M3Twitter(cache_dir="./twitter_cache", use_full_model=False, use_cuda=False, parallel=True)

08/28/2023 04:36:02 - INFO - m3inference.m3inference -   Version 1.1.5
08/28/2023 04:36:02 - INFO - m3inference.m3inference -   Running on cpu.
08/28/2023 04:36:02 - INFO - m3inference.m3inference -   Will use text model. Note that as M3 was optimized to work well with both image and text data,                                     it is not recommended to use text only model unless you do not have the profile image.
08/28/2023 04:36:03 - INFO - m3inference.m3inference -   Model text_model exists at C:\Users\An/m3/models/text_model.mdl.
08/28/2023 04:36:03 - INFO - m3inference.utils -   Checking MD5 for model text_model at C:\Users\An/m3/models/text_model.mdl
08/28/2023 04:36:03 - INFO - m3inference.utils -   MD5s match.
08/28/2023 04:36:03 - INFO - m3inference.m3inference -   Loaded pretrained weight at C:\Users\An/m3/models/text_model.mdl
